In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/df_real_token_with_embeddings_with_commas.xlsx')

In [ ]:
model_path = '/content/drive/MyDrive/model_weight_5.pth'

In [ ]:
df['임베딩'] = df['임베딩'].apply(eval)  # 쉼표가 추가된 데이터를 리스트로 변환

# GPU/CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=13)

# 학습된 모델 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

In [ ]:
# 레이블 맵 정의
label_map = {
    '민사_승소': 0, '민사_패소': 1, '민사_기각': 2,
    '형사_징역': 3, '형사_무혐의': 4, '형사_벌금': 5, '형사_기각': 6,
    '가사_승소': 7, '가사_패소': 8, '가사_기각': 9,
    '세무_승소': 10, '세무_패소': 11, '세무_기각': 12
}
reverse_label_map = {v: k for k, v in label_map.items()}

label_range = {
    '민사': [0, 1, 2],
    '형사': [3, 4, 5, 6],
    '가사': [7, 8, 9],
    '세무': [10, 11, 12]
}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 유사 사례 검색 함수
def find_top_similar_cases(input_embedding, case_type, top_n=5):
    filtered_cases = df[df['사건종류명'] == case_type]
    case_embeddings = np.stack(filtered_cases['임베딩'].dropna().values)
    similarities = cosine_similarity(input_embedding.reshape(1, -1), case_embeddings)
    top_indices = np.argsort(similarities.flatten())[-top_n:][::-1]  # 유사도 높은 순
    top_cases = filtered_cases.iloc[top_indices]
    return top_cases[['사건번호', '판결유형', '판례내용']].to_dict(orient='records')

In [ ]:
# 예측 함수
def predict(case_type, date, keyword, content):
    with torch.no_grad():
        text = f"{case_type} / {date} / {keyword} / {content}"
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model(**inputs, output_hidden_states=True)  # hidden_states 활성화
        logits = outputs.logits
        confidence = torch.softmax(logits, dim=1).max().item() * 100

        # 레이블 필터링
        possible_label_indices = [label_map[lbl] for lbl in label_map if lbl.startswith(case_type)]
        possible_logits = logits[0, possible_label_indices]
        best_index = possible_logits.argmax().item()
        best_label_index = possible_label_indices[best_index]
        best_label = reverse_label_map[best_label_index]

        final_label = best_label.split('_')[-1]

        # 유사 사례 검색
        input_embedding = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
        similar_cases = find_top_similar_cases(input_embedding, case_type, top_n=5)

        return final_label, confidence, similar_cases, text

In [ ]:
import shap
import matplotlib.pyplot as plt

# SHAP 함수
def shap_explain(content):
    # SHAP용 Forward 함수
    def model_forward(texts):
        # 텍스트를 토크나이즈하고 텐서로 변환
        inputs = tokenizer(list(texts), return_tensors="pt", max_length=512, truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        return outputs.logits.detach().cpu().numpy()

    # SHAP explainer 생성 (masker를 지정)
    explainer = shap.Explainer(model_forward, masker=shap.maskers.Text(tokenizer))

    # SHAP 값 계산
    shap_values = explainer([content])  # 사건 내용만 입력

    # SHAP 시각화
    shap.plots.text(shap_values[0])

In [ ]:
# 실행 루프
while True:
    user_input = input("사건 종류 / 사건 날짜(nnnn-nn-nn) / 사건 키워드 / 사건 내용 (종료 입력 시 종료): ")
    if user_input.lower() == '종료':
        break
    try:
        parts = user_input.split(" / ", 3)
        if len(parts) != 4:
            raise ValueError("입력이 4개의 부분으로 나누어지지 않았습니다.")
        case_type, date, keyword, content = parts
        output_label, confidence, similar_cases, text = predict(case_type, date, keyword, content)

        # 예측 결과 출력
        print(f"\n입력: {user_input}\n출력(예측 판결 유형): {output_label} (정확도: {confidence:.2f}%)")
        print("\n유사 판례:")
        for i, case in enumerate(similar_cases, 1):
            print(f"{i}. 사건 번호: {case['사건번호']}, 판결 유형: {case['판결유형']}")

        # SHAP 설명 시각화 (사건 내용만 사용)
        print("\n예측에 사용된 주요 단어 및 기여도:")
        shap_explain(content)

    except ValueError as ve:
        print(f"입력 형식 오류: {ve}\n'사건 종류 / 사건 날짜 / 사건 키워드 / 사건 내용' 형식으로 입력해주세요.")
    except Exception as e:
        print(f"오류 발생: {e}")

사건 종류 / 사건 날짜(nnnn-nn-nn) / 사건 키워드 / 사건 내용 (종료 입력 시 종료): 민사 / 2023-08-15 / 임대차 계약 / 임차인이 계약 기간 만료 전에 임의로 계약을 종료하고 임대료를 체납한 사건

입력: 민사 / 2023-08-15 / 임대차 계약 / 임차인이 계약 기간 만료 전에 임의로 계약을 종료하고 임대료를 체납한 사건
출력(예측 판결 유형): 기각 (정확도: 69.13%)

유사 판례:
1. 사건 번호: 94다31488, 판결 유형: 민사_기각
2. 사건 번호: 78다1968, 판결 유형: 민사_기각
3. 사건 번호: 94다55545, 판결 유형: 민사_기각
4. 사건 번호: 64마246, 판결 유형: 민사_기각
5. 사건 번호: 91마730, 판결 유형: 민사_기각

예측에 사용된 주요 단어 및 기여도:


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:57, 57.75s/it]               


사건 종류 / 사건 날짜(nnnn-nn-nn) / 사건 키워드 / 사건 내용 (종료 입력 시 종료): 종료


# **OUTPUT 0 ~ 12 :**
    '민사_승소': 0, '민사_패소': 1, '민사_기각': 2,
    '형사_징역': 3, '형사_무혐의': 4, '형사_벌금': 5, '형사_기각': 6,
    '가사_승소': 7, '가사_패소': 8, '가사_기각': 9,
    '세무_승소': 10, '세무_패소': 11, '세무_기각': 12
